<strong>QuickSight Incremental Deployment</strong>

Author: Ying Wang (Sr.Data Visualization Engineer in ProServe GSP)
Date: July 15 2020

Author: Vamsi Bhadriraju (Data Architect in ProServe) 
Revision Date : January 12 2021
Revision Date : May 27 2021


In [ ]:
!pip install --upgrade pip
!pip install --upgrade boto3
get_ipython().system('pip install --upgrade ipynb')

In [1]:
import boto3
import json
import time
from IPython.display import JSON
import sys
import ipynb.fs 
import logging
from typing import Any, Dict, List, Optional
from datetime import datetime

# current date and time
now = str(datetime.now().strftime("%m-%d-%Y_%H_%M"))

Import functions from functions notebook

In [2]:
from ipynb.fs.defs.Functions import data_sources
from ipynb.fs.defs.Functions import describe_source 
from ipynb.fs.defs.Functions import delete_source
from ipynb.fs.defs.Functions import create_data_source
from ipynb.fs.defs.Functions import get_datasource_name
from ipynb.fs.defs.Functions import get_datasource_ids
from ipynb.fs.defs.Functions import update_data_source_permissions

from ipynb.fs.defs.Functions import get_dataset_name
from ipynb.fs.defs.Functions import data_sets
from ipynb.fs.defs.Functions import describe_dataset
from ipynb.fs.defs.Functions import get_dataset_ids
from ipynb.fs.defs.Functions import delete_dataset 
from ipynb.fs.defs.Functions import create_dataset
from ipynb.fs.defs.Functions import update_dataset
from ipynb.fs.defs.Functions import update_data_set_permissions

from ipynb.fs.defs.Functions import get_target

from ipynb.fs.defs.Functions import templates
from ipynb.fs.defs.Functions import delete_template
from ipynb.fs.defs.Functions import update_template_permission 
from ipynb.fs.defs.Functions import copy_template
from ipynb.fs.defs.Functions import describe_template
from ipynb.fs.defs.Functions import create_template 

from ipynb.fs.defs.Functions import dashboards
from ipynb.fs.defs.Functions import describe_dashboard
from ipynb.fs.defs.Functions import create_dashboard 
from ipynb.fs.defs.Functions import delete_dashboard
from ipynb.fs.defs.Functions import update_dashboard 
from ipynb.fs.defs.Functions import get_dashboard_ids
from ipynb.fs.defs.Functions import get_dashboard_name

from ipynb.fs.defs.Functions import themes
from ipynb.fs.defs.Functions import describe_theme
from ipynb.fs.defs.Functions import delete_theme
from ipynb.fs.defs.Functions import create_theme
from ipynb.fs.defs.Functions import update_theme
from ipynb.fs.defs.Functions import describe_theme_permissions
from ipynb.fs.defs.Functions import update_theme_permissions

from ipynb.fs.defs.Functions import analysis
from ipynb.fs.defs.Functions import describe_analysis
from ipynb.fs.defs.Functions import create_analysis
from ipynb.fs.defs.Functions import delete_analysis
from ipynb.fs.defs.Functions import update_analysis
from ipynb.fs.defs.Functions import get_analysis_ids
from ipynb.fs.defs.Functions import describe_analysis_permissions



#supportive functions
from ipynb.fs.defs.Functions import data_sets_ls_of_dashboard
from ipynb.fs.defs.Functions import data_sources_ls_of_dashboard
from ipynb.fs.defs.Functions import get_data_source_deployment_list
from ipynb.fs.defs.Functions import data_sources_ls_of_analysis
from ipynb.fs.defs.Functions import data_sets_ls_of_analysis
from ipynb.fs.defs.Functions import get_user_arn
from ipynb.fs.defs.Functions import _assume_role

<strong>Static Profile</strong>

You can configure AWS profile from terminal and call the profile in below cell

In [3]:
sourceprofile=''
targetprofile=''
aws_region='us-east-1'
sourcesession = boto3.Session(profile_name=sourceprofile, region_name=aws_region)
targetsession = boto3.Session(profile_name=targetprofile, region_name=aws_region)


<strong>Assume Role</strong>

You can also assume an IAM role and create session based on the role permissions

In [ ]:
#source account
sourceaccountid=<account_id>
role_name=<role_name>
aws_region='us-east-1'
sourcesession = _assume_role(sourceaccountid, role_name, aws_region)

In [ ]:
#target account
targetaccountid=""
role_name=""
aws_region='us-east-1'
targetsession = _assume_role(targetaccountid, role_name, aws_region)
#targetsession = boto3.Session(
#        aws_access_key_id="",
#        aws_secret_access_key="",
#        aws_session_token="",
#        region_name=aws_region
#    )

<strong>Set root and admin users</strong>

root user is for the template. 
By default, we assign full permissions of objects to admin.

In [4]:
sourceroot=get_user_arn (sourcesession, 'root')
sourceadmin=get_user_arn (sourcesession, 'Administrator/wangzyn-Isengard')
#sourceversion='1'

targetroot=get_user_arn (targetsession, 'root')
targetadmin=get_user_arn (targetsession, 'Admin/wangzyn-Isengard')
#targetvpc='arn:aws:quicksight:us-east-1:889399602426:vpcConnection/sg-40b7521a'

<strong>Please define your input parameters in below cell</strong>

In [5]:
rds='mssql'
redshift={
    "ClusterId": 'wangzyncluster1',
    "Host": 'wangzyncluster1.coprq8ycemvc.us-east-1.redshift.amazonaws.com',
    "Database": 'dev'}

s3Bucket='spaceneedle-samplefiles.prod.us-east-1'
s3Key='sales/manifest.json'
vpc='sg-40b7521a'
tag=[
        {
            'Key': 'test',
            'Value': 'true'
        }
    ]
owner=targetadmin
rdscredential={
        'CredentialPair': {
            'Username': "",
            'Password': ""}}
redshiftcredential={
        'CredentialPair': {
            'Username': "ro_user",
            'Password': "Ro_user1234"}}
region='us-east-1'
namespace='default'
version='1' 

target=get_target(targetsession, rds,redshift,s3Bucket,s3Key,vpc,tag,owner,rdscredential,redshiftcredential)

JSON(target)

<IPython.core.display.JSON object>

<strong>Deployment List<strong/>

In [2]:
deploy_p = 'dashboard' 
""""
"all" will deploy data source, dataset, theme, analysis and dashboard;
"source" means data sources only; 
"dataset" means datasets only; 
"theme" means theme only;
"analysis" means analysis only;
"dashboard" means dashboard only
""" 

source_deploy_list = ["redshift-auto", "mssql", "athena_1","redshift_manual"]
dataset_deploy_list = ["patient_info"]
theme_deploy_list= ["orange"]
analysis_deploy_list= ["QuickSight_Access_Last_24_H_Analysis","Marketing Analysis"]
dashboard_deploy_list = ["QuickSight_Access_Last_24_H", "Marketing Dashboard"]

In [8]:
if deploy_p in ['dashboard']:
    source_deploy_list=[]
    dataset_deploy_list=[]
    for dashboard in dashboard_deploy_list:
        print(dashboard)
        datasources=data_sources_ls_of_dashboard(dashboard, sourcesession)
        print(datasources)
        for datasource in datasources:
            source_deploy_list.append(datasource)
        datasets=data_sets_ls_of_dashboard(dashboard, sourcesession)
        print(datasets)
        for dataset in datasets:
            dataset_deploy_list.append(dataset)
            
if deploy_p in ['analysis']:
    source_deploy_list=[]
    dataset_deploy_list=[]
    for analysis_name in analysis_deploy_list:
        print(analysis_name)
        datasources=data_sources_ls_of_analysis(analysis_name, sourcesession)
        print(datasources)
        for datasource in datasources:
            source_deploy_list.append(datasource)
        datasets=data_sets_ls_of_analysis(analysis_name, sourcesession)
        print(datasets)
        for dataset in datasets:
            dataset_deploy_list.append(dataset)
    
if deploy_p in ['all']:
    for dashboard in dashboard_deploy_list:
        datasources=data_sources_ls_of_dashboard(dashboard, sourcesession)
        for datasource in datasources:
            source_deploy_list.append(datasource)
        datasets=data_sets_ls_of_dashboard(dashboard, sourcesession)
        for dataset in datasets:
            dataset_deploy_list.append(dataset)
            
    for analysis_name in analysis_deploy_list:
        datasources=data_sources_ls_of_analysis(analysis_name, sourcesession)
        for datasource in datasources:
            source_deploy_list.append(datasource)
        datasets=data_sets_ls_of_analysis(analysis_name, sourcesession)
        for dataset in datasets:
            dataset_deploy_list.append(dataset)

QuickSight_Access_Last_24_H
['athena_1']
['quicksight_access_last_24h']
Marketing Dashboard
['redshift_manual']
['patient_info']


In [9]:
dashboard_deploy_list

['QuickSight_Access_Last_24_H', 'Marketing Dashboard']

In [10]:
dataset_deploy_list

['quicksight_access_last_24h', 'patient_info']

In [11]:
source_deploy_list

['athena_1', 'redshift_manual']

In [3]:
theme_deploy_list

['orange']

In [4]:
analysis_deploy_list

['QuickSight_Access_Last_24_H_Analysis', 'Marketing Analysis']

<strong>Results Output Location<strong/>

In [12]:
#
successlocation = "Deployment_Results/Successful/"
faillocation = "Deployment_Results/Fail/"

import os
try:
    os.makedirs(successlocation)
except OSError:
    print ("Creation of the directory %s failed" % successlocation)
else:
    print ("Successfully created the directory %s" % successlocation)

try:
    os.makedirs(faillocation)
except OSError:
    print ("Creation of the directory %s failed" % faillocation)
else:
    print ("Successfully created the directory %s" % faillocation)

Creation of the directory Migration_Results/Successful/ failed
Creation of the directory Migration_Results/Fail/ failed


<strong>deploy data sources<strong/>

In [13]:
deployment_list=get_data_source_deployment_list(sourcesession,source_deploy_list)

In [14]:
#get data sources which already deployed
targetsources=data_sources(targetsession)

#already_deployed record the data source ids of target account
already_deployed=[]
for tsource in targetsources:
    already_deployed.append(tsource['DataSourceId'])

print(already_deployed)

['18e39ce0-57a8-483b-af1f-cfa43c2bd34b']


<strong>Create data sources in target account</strong>

In [15]:
faillist=[]
newsourceslist=[]
for i in deployment_list:
    if i['DataSourceId'] not in already_deployed and 'DataSourceParameters' in i:
        print(i['DataSourceId'])
    #if 'DataSourceParameters' in i:
        newdsource=create_data_source (i, targetsession, target)
        if 'Error' in newdsource:
            faillist.append(newdsource)

        else: newsourceslist.append(newdsource)

73ebc3dd-d3b1-403f-8c99-d00b652edbf9
f1123c10-8a6a-4316-9475-a3efd163f539


<strong>Write deployment results:</strong>

In [16]:
# Datasource_Creation_Error.json records the failure on create_data_source API call step
with open(faillocation+now+'_Datasource_Creation_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

#Datasource_Creation_Fail.json records the creation_failed status data sources
#Datasource_Creation_Success.json records the successful created data sources
faillist2=[]
successfulls=[]
for news in newsourceslist:
    datasource=describe_source(targetsession, news['DataSourceId'])

    if datasource['DataSource']['Status']=="CREATION_FAILED":
        delete_source (targetsession, news['DataSourceId'])
        faillist2.append(news['DataSourceId'])

    if datasource['DataSource']['Status']=="CREATION_SUCCESSFUL":
            successfulls.append(datasource['DataSource'])

    while datasource['DataSource']['Status']=="CREATION_IN_PROGRESS":
        time.sleep(5)
        datasource=describe_source(targetsession, news['DataSourceId'])
        if datasource['DataSource']['Status']=="CREATION_SUCCESSFUL":
            successfulls.append(datasource['DataSource'])
            break
        elif datasource['DataSource']['Status']=="CREATION_FAILED":
            delete_source (targetsession, news['DataSourceId'])
            faillist2.append(news['DataSourceId'])

with open(faillocation+now+'_Datasource_Creation_Fail.json', "w") as f:
    json.dump(faillist2, f, indent=4, sort_keys=True, default=str)

with open(successlocation+now+'_Datasource_Creation_Success.json', "w") as f:
    json.dump(successfulls, f, indent=4, sort_keys=True, default=str)


In [17]:
data_sources(targetsession)

[{'Arn': 'arn:aws:quicksight:us-east-1:387046087588:datasource/73ebc3dd-d3b1-403f-8c99-d00b652edbf9',
  'DataSourceId': '73ebc3dd-d3b1-403f-8c99-d00b652edbf9',
  'Name': 'athena_1',
  'Type': 'ATHENA',
  'Status': 'CREATION_SUCCESSFUL',
  'CreatedTime': datetime.datetime(2020, 8, 12, 14, 37, 53, 530000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2020, 8, 12, 14, 37, 56, 305000, tzinfo=tzlocal()),
  'DataSourceParameters': {'AthenaParameters': {'WorkGroup': 'primary'}},
  'SslProperties': {'DisableSsl': False}},
 {'Arn': 'arn:aws:quicksight:us-east-1:387046087588:datasource/f1123c10-8a6a-4316-9475-a3efd163f539',
  'DataSourceId': 'f1123c10-8a6a-4316-9475-a3efd163f539',
  'Name': 'redshift_manual',
  'Type': 'REDSHIFT',
  'Status': 'CREATION_SUCCESSFUL',
  'CreatedTime': datetime.datetime(2020, 8, 12, 16, 38, 44, 224000, tzinfo=tzlocal()),
  'LastUpdatedTime': datetime.datetime(2020, 8, 12, 16, 38, 45, 159000, tzinfo=tzlocal()),
  'DataSourceParameters': {'RedshiftParamete

Get datasets list:

#source account
sourceaccountid=""
role_name=""
aws_region=''
sourcesession = _assume_role(sourceaccountid, role_name, aws_region)

#target account
targetaccountid=""
role_name=""
aws_region='us-east-1'
targetsession = _assume_role(targetaccountid, role_name, aws_region)

In [17]:
datasets=data_sets(sourcesession)

deployment_list=[]
for newset in dataset_deploy_list:
        ids = get_dataset_ids(newset, sourcesession)  #Get id of datasets deployment list
        for dataset in datasets:
            if ids[0] == dataset["DataSetId"]:
                deployment_list.append(dataset)

JSON(deployment_list)

<IPython.core.display.JSON object>

<strong>Get already deployd datasets list</strong>

In [19]:
#get datasets which already deployed
targetds=data_sets(targetsession)
#already_deployed record the datasets ids of target account
already_deployed=[]
for ds in targetds:
    already_deployed.append(ds['DataSetId'])
#already_deployed

deploy Datasets

In [20]:
newsetslist=[]
faillist=[]
sts_client = targetsession.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
for mds in deployment_list:
    if mds['DataSetId'] not in already_deployed:
        try:
            res=describe_dataset(sourcesession, mds['DataSetId'])
        except Exception:
            faillist.append({"Dataset": mds, "Error": str(Exception)})
            continue
   
        name=mds['Name']
        datasetid=mds['DataSetId']

        PT=res['DataSet']['PhysicalTableMap']
        for key, value in PT.items():
            for k,v in value.items():
                dsid = v['DataSourceArn'].split("/")[1]
                v['DataSourceArn']='arn:aws:quicksight:us-east-1:'+account_id+':datasource/'+dsid

        LT=res['DataSet']['LogicalTableMap']
        if 'ColumnGroups' in res['DataSet']:
            ColumnGroups=res['DataSet']['ColumnGroups']
        else: ColumnGroups=None
        try: 
            newdataset=create_dataset(targetsession, datasetid, name, PT, LT, res['DataSet']['ImportMode'], target['datasetpermission'],ColumnGroups)
            #print("new dataset: ", newdataset)
            newsetslist.append(newdataset)
        except Exception as e:
            #print('failed: '+str(e))
            faillist.append({"DataSetId": datasetid, "Name": name, "Error": str(e)})

            continue
            
    if mds['DataSetId'] in already_deployed:
        try:
            res=describe_dataset(sourcesession, mds['DataSetId'])
        except Exception:
            faillist.append({"Dataset": mds, "Error": str(Exception)})
            continue
 
        name=mds['Name']
        datasetid=mds['DataSetId']

        PT=res['DataSet']['PhysicalTableMap']
        for key, value in PT.items():
            for k,v in value.items():
                dsid = v['DataSourceArn'].split("/")[1]
                v['DataSourceArn']='arn:aws:quicksight:us-east-1:'+account_id+':datasource/'+dsid

        LT=res['DataSet']['LogicalTableMap']
        if 'ColumnGroups' in res['DataSet']:
            ColumnGroups=res['DataSet']['ColumnGroups']
        else: ColumnGroups=None
        try: 
            newdataset=update_dataset(targetsession, datasetid, name, PT, LT, res['DataSet']['ImportMode'],ColumnGroups)
            #print("new dataset: ", newdataset)
            newsetslist.append(newdataset)
        except Exception as e:
            #print('failed: '+str(e))
            faillist.append({"DataSetId": datasetid, "Name": name, "Error": str(e)})

            continue
            

In [21]:
#print fail informations
with open(faillocation+now+'Dataset_Creation_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

successfulls=[]
for news in newsetslist:
    dataset=describe_dataset(targetsession, news['DataSetId'])
    successfulls.append(dataset['DataSet'])
    
with open(successlocation+now+'Datasets_Creation_Success.json', "w") as f:
    json.dump(successfulls, f, indent=4, sort_keys=True, default=str)


Get themes list

In [ ]:
themes_list_complete =themes(sourcesession)
themes_list=[]
#JSON(datasets)
for th in themes_list_complete:
    if th["Name"] in theme_deploy_list:
        themes_list.append(th)

deploy Themes

In [ ]:
#get themes which already deployed
targetthemes=themes(targetsession)
#already_deployed record the datasets ids of target account
already_deployed=[]
for th in targetthemes:
    already_deployed.append(th['ThemeId'])
#already_deployed

In [ ]:
newthemeslist=[]
faillist=[]
sts_client = targetsession.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
for i in themes_list:
    if i['ThemeId'] not in already_deployed:
        try:
            res=describe_theme(sourcesession, i['ThemeId'])
        except Exception:
            faillist.append({"Theme": i, "Error": str(Exception)})
            continue
        THEMEID=res['Theme']['ThemeId']
        Name=res['Theme']['Name']
        BaseThemeId=res['Theme']['Version']['BaseThemeId']
        Configuration=res['Theme']['Version']['Configuration']
        try: 
            newtheme=create_theme (targetsession,THEMEID, Name,BaseThemeId,Configuration)
            newthemeslist.append(newtheme)
        except Exception as e:
            #print('failed: '+str(e))
            faillist.append({"ThemeID": THEMEID, "Name": Name, "Error": str(e)})
            continue
        try:
            update_theme_permissions(targetsession, THEMEID, targetadmin)
        except Exception as e:
            #print('failed: '+str(e))
            faillist.append({"ThemeID": THEMEID, "Name": Name, "Error": str(e)})
            continue

In [ ]:
#print fail informations
with open(faillocation+now+'Themes_Creation_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

successfulls=[]
for news in newthemeslist:
    theme=describe_theme(targetsession, news['ThemeId'])
    successfulls.append(theme['Theme']['ThemeId'])
    
with open(successlocation+now+'Themes_Creation_Success.json', "w") as f:
    json.dump(successfulls, f, indent=4, sort_keys=True, default=str)

Get analysis

In [ ]:
sourceanalysis_list_complete=analysis(sourcesession)
sourceanalysis_list=[]
for a in sourceanalysis_list_complete:
    if a["Name"] in analysis_deploy_list:
        sourceanalysis_list.append(a)

Deploy Analysis

In [ ]:
sourceanalysis_all=[]
for i in sourceanalysis_list:
    if i['Status']!= 'DELETED':
        sourceanalysis_all.append(i)

success=[]
faillist=[]
sts_client = targetsession.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
for i in sourceanalysis_all:
    sourceanalysis=describe_analysis(sourcesession, i['AnalysisId'])
    sourceanalysisid=sourceanalysis['Analysis']['AnalysisId']
    sourceanalysisArn=sourceanalysis['Analysis']['Arn']
    sourceanalysisname=sourceanalysis['Analysis']['Name']
    DataSetArns=sourceanalysis['Analysis']['DataSetArns']
    sourcetid=sourceanalysisid
    sourcetname=sourceanalysisname
    targettid=sourcetid
    targettname=sourceanalysisname
    
    TargetThemeArn=''
    if 'ThemeArn' in sourceanalysis['Analysis'].keys():
        SourceThemeArn=sourceanalysis['Analysis']['ThemeArn']
        TargetThemeArn = 'arn:aws:quicksight:'+region+':'+account_id+':theme/'+sourceanalysis['Analysis']['ThemeArn'].split("/")[1]

    sourcedsref = []
    for i in DataSetArns:
        missing=False
        did = i.split("/")[1]
        try:
            dname=get_dataset_name(did, sourcesession)
        except Exception as e:
            faillist.append({"Error Type": "Dataset: "+did+" is missing!","sourceanalysisid": sourcetid, "Name": sourcetname, "Error": str(e)})
            missing=True
            break
            
        sourcedsref.append({'DataSetPlaceholder': dname,
                    'DataSetArn': i})
    if missing: continue
    try:
        sourcetemplate = create_template(sourcesession, sourcetid, sourcetname, sourcedsref, sourceanalysisArn, '1')
        sourcetemplate=describe_template(sourcesession,sourcetid)
    except Exception as e:
        faillist.append({"Error Type": "Create Source Template Error","sourceanalysisid": sourcetid, "Name": sourcetname, "Error": str(e)})

        continue
        
    while sourcetemplate['Template']['Version']['Status']=="CREATION_IN_PROGRESS":
        time.sleep(5)
        sourcetemplate=describe_template(sourcesession,sourcetid)
        if sourcetemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            try:
                updateres=update_template_permission(sourcesession, sourcetid, targetroot)
            except Exception as e:
                delete_template(sourcesession, sourcetid)
                faillist.append({"Error Type": "Update Source Template Permission Error",
                                 "sourceanalysisid": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
    else: 
        if sourcetemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            try:
                updateres=update_template_permission(sourcesession, sourcetid, targetroot)
            except Exception as e:
                delete_template(sourcesession, sourcetid)
                faillist.append({"Error Type": "Update Source Template Permission Error",
                                 "sourceanalysisid": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
                continue          

    ds=data_sets (targetsession)
    Template=sourcetemplate['Template']
    dsref=[]
    
    missing=False
    for i in Template['Version']['DataSetConfigurations']:
    #print(i)
        n=Template['Version']['DataSetConfigurations'].index(i)
    #print(n)
        for j in ds:
            if i['Placeholder']==j['Name']:
                dsref.append({
                    'DataSetPlaceholder': i['Placeholder'],
                    'DataSetArn': j['Arn']
                })
            if n>len(dsref): 
                e="Dataset "+i['Placeholder']+"is missing!"
                faillist.append({"Error Type": "Datasets in target env are missing for this analysis",
                                 "sourceanalysisid": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
                missing=True
                break
        if missing: break
    if missing: continue
        
##working
    SourceEntity={
        'SourceTemplate': {
            'DataSetReferences': dsref,
            'Arn': Template['Arn']
        }
    }
    
    #print(SourceEntity)
    analysis=describe_analysis(targetsession, targettid)
    if 'Faild to describe analysis:' in analysis or analysis['Analysis']['Status']=='DELETED':
        if 'analysis/'+targettid+' is not found' in analysis or analysis['Analysis']['Status']=='DELETED':
            print("Create new anlaysis now:")
            try:
                newanalysis=create_analysis(targetsession, targettid, targettname,targetadmin,SourceEntity,TargetThemeArn)
            except Exception as e:
                delete_template(sourcesession, targettid)
                faillist.append({"Error Type": "Create New Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)}) 
                continue
        else:
            faillist.append({"Error Type": "Describe Target Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(analysis)}) 
            continue
    elif analysis['Analysis']['Status']=="CREATION_FAILED":
        res=delete_analysis(sourcesession, targettid)
        try:
            newanalysis=create_analysis(targetsession, targettid, targettname,targetadmin, SourceEntity,TargetThemeArn)
        except Exception as e:
            delete_template(sourcesession, targettid)
            faillist.append({"Error Type": "Create Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)})
            continue
            
    else:
        print("analysis is existing. update it now.")
        try:
            newanalysis=update_analysis(targetsession, targettid, targettname, SourceEntity,TargetThemeArn)
        except Exception as e:
            delete_template(sourcesession, targettid)
            faillist.append({"Error Type": "Update Analysis Error",
                                 "AnalysisID": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)})
            continue
    time.sleep(20)
    res=describe_analysis(targetsession,newanalysis['AnalysisId'])
    if res['Status']==200:
        status=res['Analysis']['Status']
        if status=='CREATION_SUCCESSFUL' or status=='UPDATE_SUCCESSFUL':
            success.append(res['Analysis'])
            #filename="Deployment_Results/Successful/Analysis_"+res['Analysis']['Name']+".json"
        else:
            faillist.append({"Error Type": "Analysis Creation Status is not Successful", "Analysis": res['Analysis']})
            #filename="Deployment_Results/Fail/Analysis_"+res['Analysis']['Name']+".json"

In [ ]:
with open(faillocation+now+'Analysis_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

with open(successlocation+now+'Analysis_Success.json', "w") as f:
    json.dump(success, f, indent=4, sort_keys=True, default=str)

Get dashboards list

#source account
sourceaccountid=""
role_name=""
aws_region='us-east-1'
sourcesession = _assume_role(sourceaccountid, role_name, aws_region)

#target account
targetaccountid=""
role_name=""
aws_region='us-east-1'
targetsession = _assume_role(targetaccountid, role_name, aws_region)

In [24]:
sourcedashboards=dashboards(sourcesession)

#Get id of datasets deployment list
deployment_list=[]
for newset in dashboard_deploy_list:
        ids = get_dashboard_ids(newset, sourcesession)
        for dashboard in sourcedashboards:
            if ids[0] == dashboard["DashboardId"]:
                deployment_list.append(dashboard)

JSON(deployment_list)

<IPython.core.display.JSON object>

Deploy dashboards

In [25]:
success=[]
faillist=[]
for dashboard in deployment_list:
    sourcedashboard=describe_dashboard(sourcesession, dashboard['DashboardId'])
    SourceEntityArn=sourcedashboard['Dashboard']['Version']['SourceEntityArn']
    if SourceEntityArn.split("/")[0].split(":")[-1]=="analysis":
        sourceanalysis=sourcedashboard['Dashboard']['Version']['SourceEntityArn']
    else: 
        faillist.append({"Error Type": "Source Analysis is missing!","DashboardId": sourcetid, "Name": sourcetname, "Error": "Source Analysis is missing!"})
        continue
    sourceversion=sourcedashboard['Dashboard']['Version']['VersionNumber']
    sourcedid=sourcedashboard['Dashboard']['DashboardId']
    sourcedname=sourcedashboard['Dashboard']['Name']
    sourcetid=sourcedid
    sourcetname=sourcedname
    targettid=sourcetid
    targettname=sourcedname
    DataSetArns=sourcedashboard['Dashboard']['Version']['DataSetArns']
    TargetThemeArn=''
    if 'ThemeArn' in sourcedashboard['Dashboard']['Version'].keys():
        SourceThemearn=sourcedashboard['Dashboard']['Version']['ThemeArn']
        TargetThemeArn = 'arn:aws:quicksight:'+region+':'+account_id+':theme/'+SourceThemearn.split("/")[1]
    sourcedsref = []
    #print(sourcedname)
    for i in DataSetArns:
        missing=False
        did = i.split("/")[1]
        try:
            dname=get_dataset_name(did, sourcesession)
        except Exception as e:
            faillist.append({"Error Type": "Dataset: "+did+" is missing!","DashboardId": sourcetid, "Name": sourcetname, "Error": str(e)})
            missing=True
            break
            
        sourcedsref.append({'DataSetPlaceholder': dname,
                    'DataSetArn': i})
    if missing: continue
    try:
        sourcetemplate = create_template(sourcesession, sourcetid, sourcetname, sourcedsref, sourceanalysis, '1')
        sourcetemplate=describe_template(sourcesession,sourcetid)
    except Exception as e:
        faillist.append({"Error Type": "Create Source Template Error","DashboardId": sourcetid, "Name": sourcetname, "Error": str(e)})
        continue
    
    while sourcetemplate['Template']['Version']['Status']=="CREATION_IN_PROGRESS":
        time.sleep(5)
        sourcetemplate=describe_template(sourcesession,sourcetid)
        if sourcetemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            try:
                updateres=update_template_permission(sourcesession, sourcetid, targetroot)
            except Exception as e:
                delete_template(sourcesession, sourcetid)
                faillist.append({"Error Type": "Update Source Template Permission Error",
                                 "DashboardId": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
    else: 
        if sourcetemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            try:
                updateres=update_template_permission(sourcesession, sourcetid, targetroot)
            except Exception as e:
                delete_template(sourcesession, sourcetid)
                faillist.append({"Error Type": "Update Source Template Permission Error",
                                 "DashboardId": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
                continue

    if updateres['Status']==200:
        try:
            targettemplate=copy_template(targetsession, targettid, targettname, updateres['TemplateArn'])
        except Exception as e:
            delete_template(sourcesession, sourcetid)
            faillist.append({"Error Type": "Copy Template Error",
                             "DashboardId": sourcetid, 
                             "Name": sourcetname, 
                             "Error": str(e)}) 
            continue
    else: 
        delete_template(sourcesession, sourcetid)
        faillist.append({"Error Type": "Update Source Template Permission Error",
                                 "DashboardId": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
        continue
    
    targettemplate=describe_template(targetsession,targettid)
   
    while targettemplate['Template']['Version']['Status']=="CREATION_IN_PROGRESS":
        time.sleep(5)
        targettemplate=describe_template(targetsession,targettid)
        if targettemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            break
    else: 
        if targettemplate['Template']['Version']['Status']=="CREATION_SUCCESSFUL":
            print("Template is successful copied!")
        else: 
            delete_template(targetsession, targettid)
            faillist.append({"Error Type": "Copy Template Error",
                                 "DashboardId": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
            continue
    
    ds=data_sets (targetsession)
    Template=targettemplate['Template']
    dsref=[]
    #print(Template['Version']['DataSetConfigurations'])
    missing=False
    for i in Template['Version']['DataSetConfigurations']:
        #print("i is "+str(i))
        n=Template['Version']['DataSetConfigurations'].index(i)
        #print("n is "+str(n))
        for j in ds:
            if i['Placeholder']==j['Name']:
                print(j['Name'])
                dsref.append({
                    'DataSetPlaceholder': i['Placeholder'],
                    'DataSetArn': j['Arn']
                })
                break
                print("len of dsref is "+str(len(dsref)))
                print(dsref)
        if (n+1)>len(dsref): 
            e="Dataset "+i['Placeholder']+" is missing!"
            faillist.append({"Error Type": "Datasets in target env are missing for this dashboard",
                                 "DashboardId": sourcetid, 
                                 "Name": sourcetname, 
                                 "Error": str(e)})
            missing=True
            break
        if missing: break
    if missing: continue
    #print("len of dsref is "+str(len(dsref)))
    #print(dsref)    
    SourceEntity={
        'SourceTemplate': {
            'DataSetReferences': dsref,
            'Arn': Template['Arn']
        }
    }
    #print(SourceEntity)
    dashboard=describe_dashboard(targetsession, targettid)

    if 'Faild to describe dashboard:' in dashboard:
        if 'dashboard/'+targettid+' is not found' in dashboard:
            print("Create new dashboard now:")
            try:
                newdashboard=create_dashboard(targetsession, targettid, targettname,targetadmin, SourceEntity, '1',TargetThemeArn, filter='ENABLED',csv='ENABLED', sheetcontrol='COLLAPSED')
            except Exception as e:
                delete_template(targetsession, targettid)
                faillist.append({"Error Type": "Create New Dashboard Error",
                                 "DashboardId": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)}) 
                continue
        else: 
            faillist.append({"Error Type": "Describe Target Dashboard Error",
                                 "DashboardId": targettid, 
                                 "Name": targettname, 
                                 "Error": str(dashboard)}) 
            continue
    elif dashboard['Dashboard']['Version']['Status']=="CREATION_FAILED":
        res=delete_dashboard(targetsession, targettid)
        try:
            newdashboard=create_dashboard(targetsession, targettid, targettname,targetadmin, SourceEntity, '1',TargetThemeArn, filter='ENABLED',csv='ENABLED', sheetcontrol='COLLAPSED')
        except Exception as e:
            delete_template(targetsession, targettid)
            faillist.append({"Error Type": "Create Dashboard Error",
                                 "DashboardId": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)})
            continue
            
    else:
        print("dashboard is existing. update it now.")
        try:
            res=delete_dashboard(targetsession, targettid)
            newdashboard=create_dashboard(targetsession, targettid, targettname,targetadmin, SourceEntity, '1',TargetThemeArn, filter='ENABLED',csv='ENABLED', sheetcontrol='COLLAPSED')
        except Exception as e:
            delete_template(targetsession, targettid)
            faillist.append({"Error Type": "Create Dashboard Error",
                                 "DashboardId": targettid, 
                                 "Name": targettname, 
                                 "Error": str(e)})
            continue

    res=describe_dashboard(targetsession,newdashboard['DashboardId'])
    
    if res['Status']==200:
        status=res['Dashboard']['Version']['Status']
        if status=='CREATION_SUCCESSFUL' or status=='UPDATE_SUCCESSFUL':
            success.append(res['Dashboard'])
            #filename="Deployment_Results/Successful/Dashboard_"+res['Dashboard']['Name']+".json"
        else:
            faillist.append({"Error Type": "Dashboard Creation Status is not Successful", "Dashboard": res['Dashboard']})

            #filename="Deployment_Results/Fail/Dashboard_"+res['Dashboard']['Name']+".json"

Template is successful copied!
quicksight_access_last_24h
Create new dashboard now:
Template is successful copied!
patient_info
Create new dashboard now:


In [26]:
with open(faillocation+now+'Dashboard_Error.json', "w") as f:
            json.dump(faillist, f, indent=4, sort_keys=True, default=str)

with open(successlocation+now+'Dashboard_Success.json', "w") as f:
    json.dump(success, f, indent=4, sort_keys=True, default=str)


<strong>Delete objects<strong/>

# THIS WILL DELETE ALL TARGET DATASETS

delete = "template"

if delete == "datasource":   
    for datasource in data_sources(targetsession):
        #if datasource['Type'] == "REDSHIFT":
        try:
            delete_source (targetsession, datasource['DataSourceId'])
        except Exception: pass 
elif delete == "dataset":
    for dataset in data_sets(targetsession):
        delete_dataset (targetsession, dataset['DataSetId'])
elif delete == "template":    
    for template in templates(targetsession):
        delete_template(targetsession, template['TemplateId'])
elif delete == "analysis":
    for analysis in analysis(targetsession): delete_analysis(targetsession, analysis['AnalysisId'])
    
elif delete == "dashboard":    
    for dashboard in dashboards(targetsession):
        delete_dashboard(targetsession, dashboard['DashboardId'])
delete ="don't delete anything"

Schedule notebooks to execute

https://aws.amazon.com/blogs/machine-learning/scheduling-jupyter-notebooks-on-sagemaker-ephemeral-instances/